# ML2CPP

## Preparing the dataset

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd

def populate_table(tablename, feature_names):
    iris = datasets.load_iris()
    X = iris.data  
    N = X.shape[0]
    y = iris.target.reshape(N,1)
    k = np.arange(N).reshape(N, 1)
    k_X_y = np.concatenate((k, X, y) , axis=1)
    lTable=pd.DataFrame(k_X_y)
    # print(lTable.head())
    lTable.columns = ['idx'] + feature_names + ['TGT'];
    lTable['TGT'] = lTable['TGT'].apply(int)
    lTable['idx'] = lTable['idx'].apply(int)
    lTable.to_csv(tablename , float_format='%.14g')



In [2]:
metadata = {"primary_key" : "KEY",
            "features" : ['sepal_length_cm', 'sepal_width_cm', 'petal_length_cm', 'petal_width_cm'],
            "targets" : ["TGT"],
            "table" : "iris"}

In [3]:
populate_table("/tmp/iris.csv" , metadata["features"])


In [4]:
df = pd.read_csv("/tmp/iris.csv")
df.sample(12, random_state=1960)

,Unnamed: 0,idx,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,TGT
114,114,114,5.8,2.8,5.1,2.4,2
74,74,74,6.4,2.9,4.3,1.3,1
9,9,9,4.9,3.1,1.5,0.1,0
88,88,88,5.6,3.0,4.1,1.3,1
25,25,25,5.0,3.0,1.6,0.2,0
5,5,5,5.4,3.9,1.7,0.4,0
48,48,48,5.3,3.7,1.5,0.2,0
117,117,117,7.7,3.8,6.7,2.2,2
83,83,83,6.0,2.7,5.1,1.6,1
105,105,105,7.6,3.0,6.6,2.1,2


## Training a Model

In [5]:


# train any scikit model on the iris dataset
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(df[metadata['features']].values, df[metadata['targets']].values)


/home/antoine/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


MultinomialNB()

## Deploying the Model

In [6]:

def generate_cpp_for_model(model):
    import pickle, json, requests, base64
    b64_data = base64.b64encode(pickle.dumps(model)).decode('utf-8')
    # send the model th the web service
    json_data={"Name":"model_cpp_sample", 
               "PickleData":b64_data , 
               "SQLDialect":"CPP",
               "FeatureNames" : metadata['features']}
    r = requests.post("https://sklearn2sql.herokuapp.com/model", json=json_data)
    content = r.json()
    lCPP = content["model"]["SQLGenrationResult"][0]["SQL"]
    # print(lCPP);
    return lCPP


lCPPCode = generate_cpp_for_model(clf);


In [7]:
print(lCPPCode)

namespace  {

	std::vector<std::string> get_input_names(){
		std::vector<std::string> lFeatures = { "Feature_0", "Feature_1", "Feature_2", "Feature_3" };

		return lFeatures;
	}

	std::vector<std::any> get_classes(){
		std::vector<std::any> lClasses = { 0, 1, 2 };

		return lClasses;
	}

	std::vector<std::string> get_output_names(){
		std::vector<std::string> lOutputs = { 
			"Score_0", "Score_1", "Score_2",
			"Proba_0", "Proba_1", "Proba_2",
			"LogProba_0", "LogProba_1", "LogProba_2",
			"Decision", "DecisionProba" };

		return lOutputs;
	}

	tTable compute_classification_scores(std::any Feature_0, std::any Feature_1, std::any Feature_2, std::any Feature_3) {
		auto lClasses = get_classes();

		tTable lLogLikelihoodTable = tTable();

		lLogLikelihoodTable["Score"] = {
			-1.0986122886681096 + -0.7099178212246882 * Feature_0 + -1.0867479075446411 * Feature_1 + -1.9311497334721928 * Feature_2 + -3.648801230546526 * Feature_3,
			-1.0986122886681096 + -0.8808827544295097 * Feature_0 + 

In [8]:
    def write_text_to_file(iCPPCode, oCPPFile):          
        with open(oCPPFile, "w") as text_file:
            text_file.write(iCPPCode)

    def add_cpp_main_function(iCPPCode, iCSVFile):
        lCPPCode = "#include \"Generic.i\"\n\n"
        lCPPCode = lCPPCode + iCPPCode
        lCPPCode = lCPPCode + "\tint main() {\n"
        lCPPCode = lCPPCode + "\t\tscore_csv_file(\"" + iCSVFile +"\");\n"
        lCPPCode = lCPPCode + "\treturn 0;\n}\n"
        return lCPPCode

    def compile_cpp_code_as_executable(iName):
        import subprocess
        lCommand = ["g++", "-Wall", "-Wno-unused-function", "-std=c++17" , "-g" ,  "-o", iName + ".exe",  iName + ".cpp"]
        print("EXECUTING" , "'" + " ".join(lCommand) + "'")
        result = subprocess.check_output(lCommand)
        # print(result)

    def execute_cpp_model(iName, iCSVFile):
        import subprocess
        result2 = subprocess.check_output([iName + ".exe",  iCSVFile])
        result2 = result2.decode()
        print(result2[:1000])
        print(result2[-1000:])
        return result2
        
    def execute_cpp_code(iCPPCode, iCSVFile):
        lName = "/tmp/sklearn2sql_cpp_" + str(id(clf));
        lCPPCode = add_cpp_main_function(iCPPCode, iCSVFile)
        write_text_to_file(lCPPCode, lName + ".cpp")
        compile_cpp_code_as_executable(lName)
        result = execute_cpp_model(lName, iCSVFile)
        write_text_to_file(str(result), lName + ".out")
        return lName + ".out"


In [9]:
populate_table("/tmp/iris2.csv" , ["Feature_0", "Feature_1", "Feature_2", "Feature_3"])
lCPPOutput = execute_cpp_code(lCPPCode , "/tmp/iris2.csv")
cpp_output = pd.read_csv(lCPPOutput)

EXECUTING 'g++ -Wall -Wno-unused-function -std=c++17 -g -o /tmp/sklearn2sql_cpp_140490364588528.exe /tmp/sklearn2sql_cpp_140490364588528.cpp'
idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision,DecisionProba
0,,,,0.75199536369658,0.16092948063885,0.08707515566457,-0.28502512034841,-1.82678901840572,-2.44098367502489,0,0.75199536369658
1,,,,0.68428264009004,0.19974595223750,0.11597140767246,-0.37938423020410,-1.61070895868377,-2.15441160384059,0,0.68428264009004
2,,,,0.71652350744811,0.18033184791163,0.10314464464026,-0.33334422353179,-1.71295652592662,-2.27162295898754,0,0.71652350744811
3,,,,0.66773921097677,0.20863687420353,0.12362391481970,-0.40385758440186,-1.56715998212595,-2.09051126707821,0,0.66773921097677
4,,,,0.75925580926380,0.15640082669818,0.08434336403802,-0.27541652376675,-1.85533316509091,-2.47285914483336,0,0.75925580926380
5,,,,0.71251192993341,0.18620047429031,0.10128759577629,-0.33895862321332,-1.68093136693347,-2.2897913256

In [10]:
cpp_output.sample(12, random_state=1960)

,idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision,DecisionProba
114,114,NaN,NaN,NaN,0.006520,0.437470,0.556010,-5.032835,-0.826748,-0.586969,2,0.556010
74,74,NaN,NaN,NaN,0.063763,0.502127,0.434110,-2.752583,-0.688903,-0.834456,1,0.502127
9,9,NaN,NaN,NaN,0.709821,0.185370,0.104809,-0.342742,-1.685400,-2.255619,0,0.709821
88,88,NaN,NaN,NaN,0.066601,0.492726,0.440673,-2.709040,-0.707801,-0.819453,1,0.492726
25,25,NaN,NaN,NaN,0.655516,0.217232,0.127253,-0.422333,-1.526790,-2.061581,0,0.655516
5,5,NaN,NaN,NaN,0.712512,0.186200,0.101288,-0.338959,-1.680931,-2.289791,0,0.712512
48,48,NaN,NaN,NaN,0.767120,0.152653,0.080226,-0.265112,-1.879585,-2.522904,0,0.767120
117,117,NaN,NaN,NaN,0.007017,0.479454,0.513529,-4.959451,-0.735107,-0.666448,2,0.513529
83,83,NaN,NaN,NaN,0.019483,0.479217,0.501300,-3.938220,-0.735602,-0.690550,2,0.501300
105,105,NaN,NaN,NaN,0.005246,0.464491,0.530262,-5.250199,-0.766813,-0.634383,2,0.530262


In [11]:
skl_outputs = pd.DataFrame()
X = df[metadata['features']].values
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['idx']);

skl_output_score = pd.DataFrame(columns=['Score_0', 'Score_1', 'Score_2']);
skl_output_proba = pd.DataFrame(clf.predict_proba(X), columns=['Proba_0', 'Proba_1', 'Proba_2'])
skl_output_log_proba = pd.DataFrame(clf.predict_log_proba(X), columns=['LogProba_0', 'LogProba_1', 'LogProba_2'])
skl_output_decision = pd.DataFrame(clf.predict(X), columns=['Decision'])
skl_output = pd.concat([skl_output_key, skl_output_score, skl_output_proba, skl_output_log_proba, skl_output_decision] , axis=1)
skl_output.sample(12, random_state=1960)

,idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
114,114,NaN,NaN,NaN,0.006520,0.437470,0.556010,-5.032835,-0.826748,-0.586969,2
74,74,NaN,NaN,NaN,0.063763,0.502127,0.434110,-2.752583,-0.688903,-0.834456,1
9,9,NaN,NaN,NaN,0.709821,0.185370,0.104809,-0.342742,-1.685400,-2.255619,0
88,88,NaN,NaN,NaN,0.066601,0.492726,0.440673,-2.709040,-0.707801,-0.819453,1
25,25,NaN,NaN,NaN,0.655516,0.217232,0.127253,-0.422333,-1.526790,-2.061581,0
5,5,NaN,NaN,NaN,0.712512,0.186200,0.101288,-0.338959,-1.680931,-2.289791,0
48,48,NaN,NaN,NaN,0.767120,0.152653,0.080226,-0.265112,-1.879585,-2.522904,0
117,117,NaN,NaN,NaN,0.007017,0.479454,0.513529,-4.959451,-0.735107,-0.666448,2
83,83,NaN,NaN,NaN,0.019483,0.479217,0.501300,-3.938220,-0.735602,-0.690550,2
105,105,NaN,NaN,NaN,0.005246,0.464491,0.530262,-5.250199,-0.766813,-0.634383,2


In [12]:
cpp_skl_join = skl_output.join(cpp_output , how='left', on='idx', lsuffix='_skl', rsuffix='_cpp')

In [13]:
cpp_skl_join.sample(12, random_state=1960)

,idx_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_1_cpp,Score_2_cpp,Proba_0_cpp,Proba_1_cpp,Proba_2_cpp,LogProba_0_cpp,LogProba_1_cpp,LogProba_2_cpp,Decision_cpp,DecisionProba
114,114,NaN,NaN,NaN,0.006520,0.437470,0.556010,-5.032835,-0.826748,-0.586969,...,NaN,NaN,0.006520,0.437470,0.556010,-5.032835,-0.826748,-0.586969,2,0.556010
74,74,NaN,NaN,NaN,0.063763,0.502127,0.434110,-2.752583,-0.688903,-0.834456,...,NaN,NaN,0.063763,0.502127,0.434110,-2.752583,-0.688903,-0.834456,1,0.502127
9,9,NaN,NaN,NaN,0.709821,0.185370,0.104809,-0.342742,-1.685400,-2.255619,...,NaN,NaN,0.709821,0.185370,0.104809,-0.342742,-1.685400,-2.255619,0,0.709821
88,88,NaN,NaN,NaN,0.066601,0.492726,0.440673,-2.709040,-0.707801,-0.819453,...,NaN,NaN,0.066601,0.492726,0.440673,-2.709040,-0.707801,-0.819453,1,0.492726
25,25,NaN,NaN,NaN,0.655516,0.217232,0.127253,-0.422333,-1.526790,-2.061581,...,NaN,NaN,0.655516,0.217232,0.127253,-0.422333,-1.526790,-2.061581,0,0.655516
5,5,NaN,NaN,NaN,0.712512,0.186200,0.101288,-0.338959,-1.680931,-2.289791,...,NaN,NaN,0.712512,0.186200,0.101288,-0.338959,-1.680931,-2.289791,0,0.712512
48,48,NaN,NaN,NaN,0.767120,0.152653,0.080226,-0.265112,-1.879585,-2.522904,...,NaN,NaN,0.767120,0.152653,0.080226,-0.265112,-1.879585,-2.522904,0,0.767120
117,117,NaN,NaN,NaN,0.007017,0.479454,0.513529,-4.959451,-0.735107,-0.666448,...,NaN,NaN,0.007017,0.479454,0.513529,-4.959451,-0.735107,-0.666448,2,0.513529
83,83,NaN,NaN,NaN,0.019483,0.479217,0.501300,-3.938220,-0.735602,-0.690550,...,NaN,NaN,0.019483,0.479217,0.501300,-3.938220,-0.735602,-0.690550,2,0.501300
105,105,NaN,NaN,NaN,0.005246,0.464491,0.530262,-5.250199,-0.766813,-0.634383,...,NaN,NaN,0.005246,0.464491,0.530262,-5.250199,-0.766813,-0.634383,2,0.530262


In [14]:
condition = (cpp_skl_join.Decision_cpp != cpp_skl_join.Decision_skl)
cpp_skl_join[condition]


,idx_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_1_cpp,Score_2_cpp,Proba_0_cpp,Proba_1_cpp,Proba_2_cpp,LogProba_0_cpp,LogProba_1_cpp,LogProba_2_cpp,Decision_cpp,DecisionProba
